A priori在拉丁語中指「來自以前」，被用來命名經典的挖掘資料關聯性演算法 Apriori

Apriori 採用迭代的方法先搜索出第一項集的各Item支持度，並剪去低於最小支持度的第一項集

得到第二項集後再剪去低於最小支持度的第二項集，依次類推下去直到無法找到項集為止

In [4]:
print(R.version)

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          3                           
minor          6.1                         
year           2019                        
month          07                          
day            05                          
svn rev        76782                       
language       R                           
version.string R version 3.6.1 (2019-07-05)
nickname       Action of the Toes          


In [1]:
install.packages("arules")

Warning message:
"package 'arules' is not available (for R version 3.6.1)"

In [2]:
# arules 套件包含了 Apriori 演算法在 R 中的一種實現
#install.packages("arules")
library(arules)
library(dplyr)

ERROR: Error in library(arules): there is no package called 'arules'


In [ ]:
# 想要深入了解 apriori 演算法的同學
# 建議詳閱 arules 的 vignette
# 裡面詳細介紹了 arules 提供的演算法、參數、以及範例

vignette("arules", package = "arules")

In [ ]:
# 首先請參閱 apriori 函式的 help
# 可以發現 apriori 的 data 項，必須要輸入 transactions 的型態
# 所以我們需要參閱 transaction-class 的 help
?"transaction-class"

# 從中我們可以得知，apriori 的 transaction 可以從 list、matrix 或是 data.frame 轉換而來

In [ ]:
# 舉例來說，如果我們有三種物品：`c("a", "b", "c")`
# 而第一筆交易有`c("a", "b", "c")`， 第二筆交易有`c("a","b")`
# 那我們就可以建立以下的list： record_list <- list(c("a", "b", "c"), c("a", "b"))
# 然後進而將 list 轉換為 transaction
record_list <- list(c("a", "b", "c"), c("a", "b"))
record <- as(record_list, "transactions")
record

In [ ]:
# 我們也可以用 summary 來取得更詳細的資訊
summary(record)

In [ ]:
# 我們接下來做一個範例練習
# 以下練習摘錄自 https://www.kirenz.com/post/2020-05-14-r-association-rule-mining/
# create a list of baskets
market_basket <-  
  list(  
  c("apple", "beer", "rice", "meat"),
  c("apple", "beer", "rice"),
  c("apple", "beer"), 
  c("apple", "pear"),
  c("milk", "beer", "rice", "meat"), 
  c("milk", "beer", "rice"), 
  c("milk", "beer"),
  c("milk", "pear")
  )

# set transaction names (T1 to T8)
names(market_basket) <- paste("T", c(1:8), sep = "")

In [ ]:
trans <- as(market_basket, "transactions")
dim(trans)
itemLabels(trans)
summary(trans)

In [ ]:
rules <- apriori(trans, 
                 parameter = list(supp=0.3, conf=0.5, 
                                  maxlen=10, 
                                  target= "rules"))

In [ ]:
summary(rules)

In [ ]:
inspect(rules)

In [ ]:
#Min Support 0.3, confidence as 0.5.
# 設定 minlen 以避開只有一樣 item 的 rules
rules <- apriori(trans, 
                 parameter = list(supp=0.3, conf=0.5,
                                  maxlen=10,
                                  minlen=2,
                                  target= "rules"))
inspect(rules)

In [ ]:
# 分析 顧客在買啤酒前會買什麼 設定 rhs 為 beer
beer_rules_rhs <- apriori(trans, 
                          parameter = list(supp=0.3, conf=0.5, 
                                         maxlen=10, 
                                         minlen=2),
                          appearance = list(default="lhs", rhs="beer"))
inspect(beer_rules_rhs)                          

In [ ]:
# 分析 顧客在買啤酒後會買什麼 設定 lhs 為 beer
beer_rules_lhs <- apriori(trans, 
                          parameter = list(supp=0.3, conf=0.5, 
                                         maxlen=10, 
                                         minlen=2),
                          appearance = list(lhs="beer", default="rhs"))
inspect(beer_rules_lhs)  

In [ ]:
# 嘗試不同的 support 或 confidence 參數
# 嘗試不同的 lhs 或 rhs 限制

In [ ]:
# 接著，讓我們看一個真實的範例：AdultUCI資料
# 使用下列指令載入 arules 套件隨附的 AdultUCI 資料集
data(AdultUCI)

In [ ]:
# 請參閱 AdultUCI 的 help 取得更詳細的資訊
# 在 help 中，請注意其中 example 的 部分
# 這邊講清楚了如果要將 AdultUCI 的資料轉換成可供 apriori 演算法使用的 transaction 需要經過哪些處理
# 首先我們看一下 AdultUCI 資料表的內容
dim(AdultUCI)
AdultUCI[1:2,]

In [ ]:
# 接下來進行資料處理
AdultUCI[["fnlwgt"]] <- NULL
AdultUCI[["education-num"]] <- NULL

AdultUCI[["age"]] <- ordered(cut(AdultUCI[["age"]], c(15,25,45,65,100)),
  labels = c("Young", "Middle-aged", "Senior", "Old"))

AdultUCI[["hours-per-week"]] <- ordered(cut(AdultUCI[["hours-per-week"]],
  c(0,25,40,60,168)),
  labels = c("Part-time", "Full-time", "Over-time", "Workaholic"))

AdultUCI[["capital-gain"]] <- ordered(cut(AdultUCI[["capital-gain"]],
  c(-Inf,0,median(AdultUCI[[ "capital-gain"]][AdultUCI[["capital-gain"]]>0]),
  Inf)), labels = c("None", "Low", "High"))

AdultUCI[["capital-loss"]] <- ordered(cut(AdultUCI[["capital-loss"]],
  c(-Inf,0, median(AdultUCI[["capital-loss"]][AdultUCI[["capital-loss"]]>0]),
  Inf)), labels = c("None", "Low", "High"))

# 請解釋上述資料處理在做些什麼？
# 為什麼在將 AdultUCI 資料套用 association rules 分析之前，要做這些處理？

In [ ]:
# 將 AdultUCI 轉換成為 transactions
Adult <- as(AdultUCI, "transactions")
Adult

# 各位可以直接以 example(AdultUCI) 執行上述程式碼
# 各位可以以 class、summary、str 等指令，確認 Adult 變數的結構

In [ ]:
summary(Adult)

In [ ]:
# 試回答下列問題
# 請問哪一個物品出現頻率最高？
# 每筆交易中的 itemset 長度不一，可是 AdultUCI 是一個 data.frame
# 每一列應該都有相同的欄位數，是什麼造成這種差異呢？

In [ ]:
# 接下來我們從資料集中來學習關聯性規則
rules <- apriori(Adult, parameter = list(support = 0.5, confidence = 0.9))

# 函式中的 support 與 confidence 就是設定尋找規則需要的最低 support 與最低 confidence

In [ ]:
# 這邊同樣可以藉由 summary 函式觀察找到的規則
summary(rules)

# 請問這邊一共學到多少規則？

In [ ]:
# 使用 inpsect 函式來觀看規則
inspect(rules[1])

In [ ]:
# 請以 inspect 觀察第四條規則，並解釋與第一條規則有何不同？

In [ ]:
# 這邊我們要利用 subset 函式來取出我們感興趣的規則
# 例如取出所有 rhs 包含 capircal-gain 為 None 的規則
rules.none = subset(rules, subset = rhs %in% "capital-gain=None")

In [ ]:
# 請以 inspect 觀察 rules.none 中的第一條與第二條規則

In [ ]:
# 請用 str 指令觀察 rules.none 的結構

In [ ]:
# 練習一：請用上述方法觀察 rules 資料表，並從中找出你認為最有用的規則，並解釋原因

In [ ]:
# 練習二：嘗試調整最低 support 與 confidence 產生新規則，並從中找尋有用的規則